## Initializing Pyspark

In [ ]:
import findspark
findspark.init('C:\spark-2.4.5-bin-hadoop2.7\spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf,SparkContext
from pyspark import SQLContext
sc = SparkContext(master='local',appName="My Demo")

In [ ]:
sql = SQLContext(sc)

In [ ]:
sql.read.json("D:/NYSE/output_file")

In [ ]:
#df =sql.read.json('D:/NYSE/data-master/retail_db_json/order_items')

In [ ]:
order = sc.textFile("D:/NYSE/data-master/retail_db/orders")

In [ ]:
orderItems = sc.textFile("D:/NYSE/data-master/retail_db/order_items")

In [ ]:
product = sc.textFile("D:/NYSE/data-master/retail_db/products")

## Map And FlatMap
<code>
#status =order.map((lambda status:(status.split(",")[3],1)))
linesList = ["Hey Baby","How are you","How is your mind","Do you have a good time"]
lines = sc.parallelize(linesList)
wdcnt = lines.flatMap(lambda a:a.split(" "))
from operator import add
result = wdcnt.map(lambda a:(a,1))
result.reduceByKey(lambda a,b:a+b).collect()
</code>

## Filter in PySPark
<code>
#Boolean ondition and Filter Use in Data
completedOrder = order.filter(lambda a:a.split(",")[3]=="COMPLETE")
completedOrder = order\
.filter(lambda a:(a.split(",")[3] in ["COMPLETE" ,"CLOSED"] and a.split(',')[1].split(" ")[0][0:7]=="2013-07"))
</code>

## Join in PySpark
<code>
orderInPair = order.map(lambda a:(int(a.split(",")[0]),a.split(",")[3]))
itemsInPair= orderItems.map(lambda a:(int(a.split(",")[1]),float(a.split(",")[4])))
rightDF.filter(lambda o:o[1][0]==None).take(10)
joinedDf.filter(lambda o:o[1][1]==None).take(10)
fullouterJoin = orderInPair.fullOuterJoin(itemsInPair)
joinedDfinner = orderInPair.join(itemsInPair)
joinedDf = orderInPair.leftOuterJoin(itemsInPair)
rightDF = itemsInPair.rightOuterJoin(orderInPair)
</code>

## Aggregation in PySpark
<code>
from operator import add
secStage = orderItems.map(lambda a:(int(a.split(",")[1]),float(a.split(",")[4])))
minOde =orderItems.filter(lambda a:int(a.split(",")[1])==2)
result = minOde.\
reduce(lambda a,b:
       a if (float(a.split(",")[4]) < float(b.split(",")[4])) else b
            )
orderStatus =order.\
map(lambda o: (o.split(",")[3],1))
countByStatus =orderStatus.countByKey()# Not Suitable because after that again we need to convert into RDD


### GroupByKey()- Two use case 1) sum of revenue 2) sort in Descending order
orMap=orderItems.map(lambda oi:(int(oi.split(",")[1]),float(oi.split(",")[4])))
df =orMap.groupByKey()
df.map(lambda oi:(oi[0],round(sum(oi[1]),2))).take(5)

orMapw =orderItems.map(lambda oi:(int(oi.split(",")[1]),oi))
ff =orMapw.groupByKey()
fd =ff.flatMap(lambda oo : (sorted(oo[1],key = lambda k:float(k.split(",")[4]),reverse=True)))

#RduceByKey
rbk = orderItems.map(lambda oi: (int(oi.split(",")[1]),oi))
from operator import add
rbk.reduceByKey(add)
rbk.reduceByKey(lambda a,b: a if(float(a.split(",")[4])<float(a.split(",")[4])) else b)

#### AggregateByKey(2 lambda function and scopeValue)- Revenue and count(Combiner and reducer logic different)
revMap =orderItems.map(lambda oi:(int(oi.split(",")[1]),float(oi.split(",")[4])))
revenueCount = revMap.aggregateByKey((0.0,0),lambda a,b:(a[0]+b,a[1]+1),
                                     lambda x,y: (x[0]+y[0],x[1]+y[1]))
</code>
#####  Never use GroupBy unless ReduceByKey or AggregateByKey is enough for a problem to solve 


## Sorting in PySPark
<code>
df =product.filter(lambda a:a.split(",")[4] != "")\
   .map(lambda a:(float(a.split(",")[4]),a))
resultHelper =df.sortByKey(True)
result = resultHelper.map(lambda a:a[1])

dfn =product.filter(lambda p:p.split(",")[4] != "")\
   .map(lambda a:((int(a.split(",")[1]),float(a.split(",")[4])),a))
#### Tricks 
dfv =product.filter(lambda p:p.split(",")[4] != "")\
   .map(lambda a:((int(a.split(",")[1]),-float(a.split(",")[4])),a))
resultHelp =dfv.sortByKey()# sort by one ascending another decnding by appending "-" before the data in DESC
result = resultHelp.map(lambda a:a[1])
</code>

## Ranking and its power in Pyspark

<code>
new =product.filter(lambda p:p.split(",")[4] != "")\
   .map(lambda a:(float(a.split(",")[4]),a))
   
sortByPrice = new.sortByKey(False)
sortByPrice.map(lambda a:a[1])
productFilter.top(5,key=lambda a:float(a.split(',')[4]))#top function
productFilter.takeOrdered(5,key=lambda a:-float(a.split(',')[4]))#takeOrdered
df =productFilter.map(lambda a: (int(a.split(",")[1]),a))
l=df.groupByKey()
result =l.flatMap(lambda a:sorted(a[1],key=lambda k: float(k.split(",")[4]),reverse=True)[:3])

df =productFilter.map(lambda a: (int(a.split(",")[1]),a))
l=df.groupByKey()
newSOrt =sorted(filterd[1], key=lambda a:float(a.split(",")[4]),reverse=True)
l_Map=map(lambda a:round(float(a.split(",")[4]),2),newSOrt)
topNPrices =sorted(set(l_Map),reverse=True)[:3]
import itertools as it
topNPricedProduct =it.takewhile(lambda p:float(p.split(",")[4]) in topNPrices,newSOrt)
<code>

## GroupBy with complecated logic 
<code>
productFilter=product.filter(lambda a:a.split(",")[4] != "")
df =productFilter.map(lambda a: (int(a.split(",")[1]),a))
groBy =df.groupByKey()
filtr =groBy.filter(lambda a:a[0]==59)
def getTopNPriceswithDUp(productOnCategoryId, topN):
    newSOrt =sorted(productOnCategoryId[1],
                    key=lambda a:float(a.split(",")[4])
                    ,reverse=True)
    l_Map=map(lambda a:round(float(a.split(",")[4]),2),newSOrt)
    topNPrices =sorted(set(l_Map),reverse=True)[:topN]
    print(topNPrices)
    import itertools as it
    return it.takewhile(lambda p:
                        float(p.split(",")[4]) in topNPrices,
                        newSOrt
                        )
topNPriceProduct = groBy.flatMap(lambda p:getTopNPriceswithDUp(p,3))#All duplicate + first 3 top rate
</code>

## Set Operation in PySpark RDD for Transformation
<code>
ord201312 = order\
.filter(lambda a: a.split(",")[1][:7]=='2013-12').\
map(lambda a:(int(a.split(",")[0]),a))
ord201401 = order.filter(lambda a: a.split(",")[1][:7]=='2014-01')\
     .map(lambda a: (int(a.split(",")[0]), a))
          
orderItemsMap = orderItems.map(lambda a:(int(a.split(",")[1]),a)) 

ordersJoin201312 = ord201312.join(orderItemsMap)
ordersJoin201401 = ord201401.join(orderItemsMap)

df =ordersJoin201312.map(lambda a: a[1][1])
ddf = ordersJoin201401.map(lambda a: a[1][1])

product201312 = df.map(lambda p:int(p.split(",")[2]))
product201401 = ddf.map(lambda p:int(p.split(",")[2]))
product201312.union(product201401).distinct()
commonProducts =product201312.intersection(product201401)#automatically distinct is applied

onlyIn201312 =product201312.subtract(product201401).distinct()
onlyIn201401 =product201401.subtract(product201312).distinct()

onlyInOneMonth =onlyIn201312.union(onlyIn201401)
</code>

## Data Saving
<code>
revHelper =orderItems.map(lambda a : (int(a.split(",")[1]),float(a.split(",")[4])))

revNue = revHelper.reduceByKey(lambda a,b:round(a+b,2))
resultFinal =revNue.map(lambda t:str(t[0])+"\t"+str(t[1]))
###  ticks
##codecs in /etc/hadoop/conf/core-site.xml for Compression
resultFinal.saveAsTextFile("D:/NYSE/output_file",compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")
dfTest = sc.textFile("D:/NYSE/output_file")# Read Compress Data with supported algorithm will able to read the data automatically
</code>

#### Need data to be save as Data Frame to save it as ORC,Parquet,JSON,avro(databricks)

## Data Save after transformationin PySPark

<code>
revHelper =orderItems.map(lambda a : (int(a.split(",")[1]),float(a.split(",")[4])))
revNue = revHelper.reduceByKey(lambda a,b:round(a+b,2))
dfResult =revNue.toDF(schema=["order_id","order_revenue"])
dfResult.write.save("D:/NYSE/output_file","json","snappy")
##### Only for Text file only need to care for delimiter for other no needed
#### dfResult.write.csv("D:/NYSE/output_file")    
</code>